<a href="https://www.kaggle.com/code/aisuko/wikipedia-q-a-retrieval-semantic-search?scriptVersionId=162281766" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

We show two **symmetric search task** in [Similar Questions Retrieval](https://www.kaggle.com/code/aisuko/similar-questions-retrieval) and [Semantic Search in Publications](https://www.kaggle.com/code/aisuko/semantic-search-in-publications) notebooks. In this notebook, we are going to show a **asymmetric search task**.

We will use model `nq-distilbert-base-v1` which was trained on the Natural Questions dataset. It consists of about 100k real Google search queries, together with an annotated passage from Wikipedia that provides the answer. 

As corpus, we use the similar Simple English Wikipedia so that it fits easily into memory.

In [1]:
!pip install sentence-transformers==2.3.1
!pip install datasets==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 

In [2]:
import os

os.environ['MODEL_NAME']='nq-distilbert-base-v1'
os.environ['DATASET_NAME']='simplewiki-2020-11-01.jsonl.gz'
os.environ['EMBEDDINGS_NAME']='simplewiki-2020-11-01-nq-distilbert-base-v1.pt'
os.environ['EMBEDDINGS_URL']='http://sbert.net/datasets/simplewiki-2020-11-01-nq-distilbert-base-v1.pt'
os.environ['DATASET_URL']='http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz'

# Loading the dataset

AS the dataset, we use Simple English Wikipedia. Compared to the full English Wikipedia, it has only about 170k articles. We split these articles into paragraphs and encode them with our semantic models. 

In [3]:
import gzip
import json
from sentence_transformers.util import http_get

# download the dataset first
http_get(os.getenv('DATASET_URL'), os.getenv('DATASET_NAME'))

passages=[]
with gzip.open(os.getenv('DATASET_NAME'), 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data=json.loads(line.strip())
        for paragraph in data['paragraphs']:
            # We encode the passages as [title, text]
            passages.append([data['title'], paragraph])
len(passages)

  0%|          | 0.00/50.2M [00:00<?, ?B/s]

509663

# Loading the embeddings

Here we download the embeddings directly from datasets. It would take 30 mins on single GPU P100. See detail [simple_english_wikipedia](https://huggingface.co/datasets/aisuko/simple_english_wikipedia).

In [4]:
from datasets import load_dataset

embedding_ds=load_dataset('aisuko/simple_english_wikipedia')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:765: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


In [5]:
import torch

corpus_embeddings=torch.from_numpy(embedding_ds['train'].to_pandas().to_numpy()).to(torch.float)
len(corpus_embeddings)

509663

In [6]:
# Loading/Computing the embeddings
# We can load the pre-computed embeddings or compute it directly.

# import torch

# load_pre_computed=False

# if load_pre_computed:
#     http_get(os.getenv('EMBEDDINGS_URL'), os.getenv('EMBEDDINGS_NAME'))
#     corpus_embeddings=torch.load(os.getenv('EMBEDDINGS_NAME'))
#     corpus_embeddings=corpus_embeddings.float().to('cuda')

# Loading model

In [7]:
from sentence_transformers import SentenceTransformer

model=SentenceTransformer(os.getenv('MODEL_NAME'))
model.max_seq_length=256
model.to('cuda')
model

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/540 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
)

In [8]:
# # It tooks 11min on Single GPU P100
# from sentence_transformers.util import normalize_embeddings
# if not load_pre_computed:
#     corpus_embeddings=model.encode(passages, convert_to_tensor=True, show_progress_bar=True).to('cuda')
#     corpus_embeddings=normalize_embeddings(corpus_embeddings)

# corpus_embeddings

In [9]:
# It takes too much time to save to csv
# import pandas as pd

# embedding_data=pd.DataFrame(corpus_embeddings.cpu())
# embedding_data.to_csv('simple_english_wikipedia.csv', index=False)

# Define the search function

In [10]:
import time
from sentence_transformers.util import semantic_search

def search(query, top_k=5):
    # Encode the query using the model and find potentially relevant passages
    start_time=time.time()
    question_embedding=model.encode(query, convert_to_tensor=True).to('cuda')
    hits=semantic_search(
        question_embedding,
        corpus_embeddings,
        query_chunk_size=100,
        corpus_chunk_size=500000,
        top_k=top_k
    )
    hits=hits[0]
    end_time=time.time()
    
    # Output of top-k hits
    print('Input question', query)
    print('Results (after {:.3f} seconds):'.format(end_time-start_time))
    for hit in hits:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']]))

# Inference

In [11]:
search(query='What is the capital of the Australia?')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Input question What is the capital of the Australia?
Results (after 2.102 seconds):
	0.771	['Australia', 'Australia, formally the Commonwealth of Australia, is a country and sovereign state in the southern hemisphere, located in Oceania. Its capital city is Canberra, and its largest city is Sydney.']
	0.707	['New South Wales', 'New South Wales is one of the states of Australia. It the oldest state in Australia and is sometimes called the "Premier State". Of all Australian states, New South Wales has the most people. An inhabitant of New South Wales is referred to as a New South Welshman. The capital city of New South Wales is Sydney. Sydney is the biggest city in Australia.']
	0.687	['Australia', "In 2013 according to world bank Australia had just over 23.13 million people. Most Australians live in cities along the coast, such as Sydney, Melbourne, Brisbane, Perth, Darwin, Hobart and Adelaide. The largest inland city is Canberra, which is also the nation's capital. The largest city is 